# Curation Estaciones Meteorologicas

Estudiar La cantidad de datos validos disponibles para cada variable en cada estacion y obtener el numero de estaciones con los datos minimos necesarios para el estudio

In [1]:
setwd("~/Repositories/AirQualityCOVID")

In [2]:
source("src/general.R")

## Funciones para filtrar datos

In [3]:
suppressMessages(library(lubridate))
suppressMessages(library(tidyverse))
suppressMessages(library(plyr))

In [4]:
get.info <- function(st) {
    
    dataframe <- read.csv(paste("data/Curation/AEMET/Values/",
                                st,
                                ".csv", sep=""))
    
    dataframe$fecha <- ymd(dataframe$fecha)
    
    cbind(data.frame(site=levels(dataframe$siteAQ),
                     start_dt=min(dataframe$fecha),
                     end_dt=max(dataframe$fecha)
                    ),
          rbind(apply(dataframe, 2, 
                      function(df) {
                          (sum(!is.na(df)) / length(df))
                      }))
         )
}

In [5]:
sitesAQ <- read.csv("data/Curation/sitesAQ.csv")

all.data <- do.call(rbind.fill, lapply(levels(sitesAQ$site), get.info))

# Guardar Datos

```R
write.csv(all.data,
          "data/Curation/AEMET/info_sites.csv",
          row.names=F)
```

In [26]:
a <- rbind(apply(all.data, 2, 
            function(df){
                print(df[1:5])
                mean(df>= 0.9, na.rm=TRUE)
            }))

[1] "es0041a" "es0110a" "es0115a" "es0118a" "es0120a"
[1] "2013-01-01" "2013-01-01" "2013-01-01" "2013-01-01" "2013-01-01"
[1] "2020-12-31" "2020-12-31" "2020-12-31" "2020-12-31" "2020-12-31"
[1] "1" "1" "1" "1" "1"
[1] "1" "1" "1" "1" "1"
[1] "1" "1" "1" "1" "1"
[1] "1" "1" "1" "1" "1"
[1] "1" "1" "1" "1" "1"
[1] "1.0000000" "1.0000000" "0.9993158" "0.9993158" "0.9261626"
[1] "1.0000000" "1.0000000" "1.0000000" "1.0000000" "0.9708910"
[1] "1.0000000" "1.0000000" "0.9993158" "0.9993158" "0.9261626"
[1] " 1" " 1" " 1" " 1" " 1"
[1] "1.0000000" "1.0000000" "0.9993158" "0.9993158" "0.9261626"
[1] " 1" " 1" " 1" " 1" " 1"
[1] "0.9996579" "0.9996579" "0.9418406" "0.9418406" "0.5012425"
[1] "0.9993158" "0.9993158" "0.9483407" "0.9483407" "0.5136670"
[1] "0.9996579" "0.9996579" "0.9418406" "0.9418406" "0.5012425"
[1] " 1" " 1" " 1" " 1" " 1"
[1] "1.0000000" "1.0000000" NA          NA          NA         
[1] "1.0000000" "1.0000000" "0.9846049" "0.9846049" NA         
[1] "1.0000000" "1.000000

In [16]:
sum(all.data$horaracha >= 0.9, na.rm=T)

[1] 60

In [27]:
data.frame(a)$horaracha

[1] 0

In [28]:
a

site,start_dt,end_dt,fecha,indicativo,nombre,provincia,altitud,tmed,prec,⋯,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,siteAQ
1,1,1,1,1,1,1,1,0.9850746,1,⋯,0.9,0.9,0.9,0,0.9038462,1,1,1,1,1
